# Object Speed Estimation

In [ ]:
import cv2
import numpy as np
import sort

# Neural Network

using pre-trained neural network model yolov7-tiny.

In [ ]:
neural_network = cv2.dnn_DetectionModel('yolov7-tiny.weights', 'yolov7-tiny.cfg')
neural_network.setInputSize((640, 640))
neural_network.setInputScale(1.0/255.0)
neural_network.setInputSwapRB(True)

In [ ]:
class_list = []
with open('coco.names') as f:
    class_list = f.read().split('\n')

In [ ]:
def infobox_over_object(frame: np.ndarray, box: list, text: str, box_color: tuple = (255, 100, 40), text_color: tuple = (255, 255, 255)):
    """
    Draws box over and object in the "frame" and puts text above the drawn box over the object.

    frame is opencv frames (numpy).

    box is clockwise starting from left top x_cord , y_cord together. (8 members in list)
    """
    cv2.rectangle(frame, (box[0]-1, box[1]),
                  (box[0]+box[2]+1, box[1]+box[3]), box_color, 2)
    cv2.rectangle(frame, (box[0]-2, box[1] - 18),
                  (box[0]+box[2]+2, box[1]), box_color, -1)
    cv2.putText(frame, text, (box[0], box[1] - 4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1)

In [ ]:
# cam = cv2.VideoCapture('MOT20-01-raw.webm')
cam = cv2.VideoCapture('videoplayback.mp4')
s = sort.SORT()

while True:
    ret_status, frame = cam.read()
    if not ret_status:
        break

    classes, confidences, boxes = neural_network.detect(frame, confThreshold=0.5, nmsThreshold=0.6)

    # print(confidences.reshape(-1, 1)*100)
    # print(boxes)
    model_predictions = np.concatenate((boxes, 100 - confidences.reshape(-1, 1)*100), axis=1, dtype=np.int32, casting='unsafe')
    print(model_predictions)
    res = s.update(model_predictions)

    for i in res:
        infobox_over_object(frame, i[:4], str(i[5]))
    
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(0)

    if key == ord('q'):
        break

cv2.destroyAllWindows()
cam.release()